In [2]:
# 0. Select GPU in Colab: Runtime > Change runtime type > GPU
# Then run this cell to install dependencies
!pip install -U ultralytics
!pip install opencv-python-headless==4.7.0.72
!pip install git+https://github.com/ifzhang/ByteTrack.git@master  # bytetrack python (may take a minute)
# (Optional) Labellerr SDK if you want to upload predictions (replace URL if different)
!pip install https://github.com/tensormatics/SDKPython/releases/download/prod/labellerr_sdk-1.0.0.tar.gz || true


  Cloning https://github.com/ifzhang/ByteTrack.git (to revision master) to /tmp/pip-req-build-l2_4_rxx
  Running command git clone --filter=blob:none --quiet https://github.com/ifzhang/ByteTrack.git /tmp/pip-req-build-l2_4_rxx
  Running command git checkout -q master
  error: pathspec 'master' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q master did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q master did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... done


In [2]:
import os
import zipfile
import json
from google.colab import files

# 1️⃣ Upload images.zip and instances_train.json
uploaded = files.upload()

# 2️⃣ Extract images.zip
for filename in uploaded.keys():
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("/content/dataset")

# 3️⃣ Move/define paths
dataset_dir = "/content/dataset"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

json_path = "instances_train.json"  # uploaded separately

# 4️⃣ Load JSON file
with open(json_path, "r") as f:
    coco_data = json.load(f)

print("Categories:", coco_data["categories"])
print("Train images in JSON:", len(coco_data["images"]))
print("Train annotations:", len(coco_data["annotations"]))
print("Train images on disk:", len(os.listdir(train_dir)))
print("Test images on disk:", len(os.listdir(test_dir)))


Saving images.zip to images.zip
Saving instances_train.json to instances_train.json
Categories: [{'id': 0, 'name': 'Vehicle', 'supercategory': 'Annotation', 'isthing': 1, 'color': [0, 0, 255], 'attributes': [], 'labellerr_question_id': 'b9094a34-dd43-4e85-8b7d-c0e575e265c1'}, {'id': 1, 'name': 'Pedestrians', 'supercategory': 'Annotation', 'isthing': 1, 'color': [255, 0, 0], 'attributes': [], 'labellerr_question_id': 'q-objects-001'}]
Train images in JSON: 100
Train annotations: 484
Train images on disk: 100
Test images on disk: 50


In [5]:
# ===============================
# 3️⃣ Convert COCO JSON → YOLOv8 Segmentation format
# ===============================

from pycocotools.coco import COCO
from tqdm import tqdm
import shutil

save_img_dir = "/content/yolo_dataset/train/images"
save_lbl_dir = "/content/yolo_dataset/train/labels"
os.makedirs(save_img_dir, exist_ok=True)
os.makedirs(save_lbl_dir, exist_ok=True)

# Copy images to YOLO folder
for img in os.listdir(train_dir):
    shutil.copy(os.path.join(train_dir, img), os.path.join(save_img_dir, img))

# Load COCO
coco = COCO(json_path)

# Convert each image’s annotations
for img in tqdm(coco.imgs.values()):
    img_id = img["id"]
    img_w, img_h = img["width"], img["height"]

    ann_ids = coco.getAnnIds(imgIds=[img_id])
    anns = coco.loadAnns(ann_ids)

    file_name = os.path.splitext(img["file_name"])[0] + ".txt"
    with open(os.path.join(save_lbl_dir, file_name), "w") as f:
        for ann in anns:
            cat = ann["category_id"] - 1  # YOLO expects 0-based category ids

            # segmentation polygons
            if "segmentation" in ann and isinstance(ann["segmentation"], list):
                for seg in ann["segmentation"]:
                    # Normalize polygon points
                    poly = []
                    for i in range(0, len(seg), 2):
                        x = seg[i] / img_w
                        y = seg[i+1] / img_h
                        poly.extend([x, y])

                    # YOLOv8 format: class x1 y1 x2 y2 ... xn yn
                    f.write(f"{cat} " + " ".join([str(p) for p in poly]) + "\n")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 100/100 [00:00<00:00, 2960.43it/s]


In [6]:
# ===============================
# 4️⃣ Create data.yaml for YOLOv8
# ===============================

yaml_path = "/content/yolo_dataset/data.yaml"

class_names = [cat["name"] for cat in coco_data["categories"]]

yaml_text = f"""
train: /content/yolo_dataset/train/images
val: {test_dir}   # unlabeled test images (used just for predictions)

nc: {len(class_names)}
names: {class_names}
"""

with open(yaml_path, "w") as f:
    f.write(yaml_text)

print("✅ data.yaml created:")
print(yaml_text)


✅ data.yaml created:

train: /content/yolo_dataset/train/images
val: /content/dataset/test   # unlabeled test images (used just for predictions)

nc: 2
names: ['Vehicle', 'Pedestrians']



In [4]:
# 🔧 Fix NumPy compatibility issues (Colab currently ships numpy 2.x)
!pip install "numpy<1.26" --force-reinstall
!pip install --upgrade --force-reinstall ultralytics opencv-python-headless==4.10.0.84 pycocotools tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 89.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Using cached ultralytics-8.3.203-py3-none-any.whl.metadata (37 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.1 MB/s eta 0:00:00
  Using cached ultralytics_thop-2.0.17-py3-none-any

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model = YOLO("yolov8s-seg.pt")

model.train(
    data="/content/yolo_dataset/data.yaml",
    epochs=100,
    imgsz=640,
    device=0
)


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:850: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
      2/100      2.83G     0.7543      4.836       3.49     0.9361          8        640: 100% ━━━━━━━━━━━━ 1/1 5.2it/s 0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 4.6it/s 0.4s
                   all         50          0          0          0          0          0          0          0          0          0
WARNING ⚠️ no labels found in segment set, can not compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:850: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
      3/100      2.83G     0.5692      5.313      2.633     0.9198         15        640: 100% ━━━━━━━━━━━━ 1/1 7.1it/s 0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 4.5it/s 0.4s
                   all         50          0          0          0          0          0          0          0          0          0
WARNING ⚠️ no labels found in segment set, can not compute metrics without labels


...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
     47/100      2.99G     0.7154      1.211      1.123      0.875          8        640: 100% ━━━━━━━━━━━━ 1/1 6.3it/s 0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 5.1it/s 0.4s
                   all         50          0          0          0          0          0          0          0          0          0
WARNING ⚠️ no labels found in segment set, can not compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:850: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
     48/100      2.99G     0.7674       1.21      1.004     0.9734         16        640: 100% ━━━━━━━━━━━━ 1/1 8.1it/s 0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 5.5it/s 0.4s
                   all         50          0          0          0          0          0          0          0          0          0
WARNING ⚠️ no labels found in segment set, can not compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:850: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


KeyboardInterrupt: 

In [7]:
import json
import os
import numpy as np
from tqdm import tqdm  # optional, shows progress

# ---------------- CONFIG ----------------
json_file = "/content/instances_train.json"  # your Labellerr JSON
images_dir = "/content/dataset/train"                # folder with training images
yolo_labels_dir = "labels"           # folder to save YOLO .txt labels
class_mapping = {
    -1: 0,  # vehicle
    -2: 1   # pedestrian (or whatever your Labellerr IDs are)
}

# ---------------------------------------

os.makedirs(yolo_labels_dir, exist_ok=True)

# Load JSON
with open(json_file, "r") as f:
    data = json.load(f)

# Helper function: polygon -> YOLO bbox
def polygon_to_yolo_bbox(polygon, img_width, img_height):
    """
    Converts polygon points to YOLO bbox format:
    [x_center, y_center, width, height] normalized to [0,1]
    """
    pts = np.array(polygon).reshape(-1, 2)
    x_min, y_min = pts.min(axis=0)
    x_max, y_max = pts.max(axis=0)
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return [x_center, y_center, width, height]

# Convert each annotation
for ann in tqdm(data["annotations"], desc="Converting annotations"):
    img_id = ann["image_id"]
    image_info = next(img for img in data["images"] if img["id"] == img_id)
    width, height = image_info["width"], image_info["height"]

    # Labellerr polygons are under "segmentation"
    if "segmentation" not in ann or not ann["segmentation"]:
        continue

    # Convert polygon -> YOLO bbox
    yolo_bbox = polygon_to_yolo_bbox(ann["segmentation"][0], width, height)

    # Map class
    class_id = class_mapping.get(ann.get("category_id", -1), 0)

    # Save to YOLO .txt file (same name as image)
    txt_filename = os.path.join(
        yolo_labels_dir,
        image_info["file_name"].rsplit(".", 1)[0] + ".txt"
    )
    with open(txt_filename, "a") as f:
        f.write(f"{class_id} " + " ".join([str(x) for x in yolo_bbox]) + "\n")

print("✅ Conversion complete! YOLO labels are in:", yolo_labels_dir)


Converting annotations: 100%|██████████| 484/484 [00:00<00:00, 16824.77it/s]

✅ Conversion complete! YOLO labels are in: labels


In [8]:
import os
import shutil
from sklearn.model_selection import train_test_split

# ---------------- CONFIG ----------------
images_dir = "/content/dataset/train"  # your original images
labels_dir = "/content/labels"         # your YOLO .txt labels
dataset_dir = "/content/dataset_yolo"  # new YOLO dataset folder
val_split = 0.2                        # fraction for validation
# ---------------------------------------

# Create YOLO folder structure
for split in ["train", "val"]:
    os.makedirs(os.path.join(dataset_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, "labels", split), exist_ok=True)

# Get all images that have labels
all_images = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png"))]
images_with_labels = [f for f in all_images if os.path.exists(os.path.join(labels_dir, f.rsplit(".",1)[0]+".txt"))]

# Split into train and validation
train_imgs, val_imgs = train_test_split(images_with_labels, test_size=val_split, random_state=42)

# Function to copy images and labels
def copy_files(file_list, split):
    for img_file in file_list:
        base_name = img_file.rsplit(".",1)[0]
        label_file = base_name + ".txt"

        # Copy image
        shutil.copy(os.path.join(images_dir, img_file),
                    os.path.join(dataset_dir, "images", split, img_file))

        # Copy label
        shutil.copy(os.path.join(labels_dir, label_file),
                    os.path.join(dataset_dir, "labels", split, label_file))

# Copy files
copy_files(train_imgs, "train")
copy_files(val_imgs, "val")

print("✅ Dataset organized for YOLO!")
print(f"Train images: {len(train_imgs)}, Val images: {len(val_imgs)}")
print("Folder structure inside:", dataset_dir)


✅ Dataset organized for YOLO!
Train images: 80, Val images: 20
Folder structure inside: /content/dataset_yolo


In [9]:
import yaml

# ---------------- CONFIG ----------------
dataset_dir = "/content/dataset_yolo"  # path from previous step
num_classes = 1                         # you only have 1 class (-1 mapped to 0)
class_names = ["object"]                # replace with your actual class name
yaml_file = "/content/data.yaml"        # output YAML file path
# ---------------------------------------

data_yaml = {
    'train': os.path.join(dataset_dir, "images/train"),
    'val': os.path.join(dataset_dir, "images/val"),
    'nc': num_classes,
    'names': class_names
}

# Save YAML
with open(yaml_file, 'w') as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml created at:", yaml_file)


✅ data.yaml created at: /content/data.yaml


In [10]:
!pip install ultralytics


In [12]:
from ultralytics import YOLO

# Load YOLOv8 model (you can use 'yolov8n.pt' for nano, 'yolov8s.pt' for small, etc.)
model = YOLO("yolov8n.pt")

# Train
model.train(
    data="/content/dataset_yolo/data.yaml",  # path to your data.yaml
    epochs=100,                               # adjust as needed
    imgsz=640,                               # image size
    batch=8,                                 # batch size
    name="vehicle_pedestrian_yolov8"         # experiment name
)


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_yolo/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=vehicle_pedestrian_yolov82, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e3ba95d7ad0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [20]:

# 🔹 Step 1: Import YOLO
from ultralytics import YOLO
import os

# Path to your trained YOLO-Seg model
model_path = '/content/runs/segment/train/weights/best.pt'

# Load the model
model = YOLO(model_path)

# 🔹 Step 2: Set paths
test_images_dir = '/content/dataset/test/'    # Folder containing your test images
output_dir = '/content/runs/predict/test_results/'

os.makedirs(output_dir, exist_ok=True)

# 🔹 Step 3: Run inference
results = model.predict(
    source=test_images_dir,  # Can be a folder, single image, or video
    save=True,               # Save images with predicted masks overlaid
    save_txt=False,          # Don't save YOLO txt annotations
    imgsz=640,               # Resize images for inference (optional)
    conf=0.25                # Confidence threshold for predictions
)

# 🔹 Step 4: Show output folder location
print(f"Predicted masks and overlayed images saved at: {output_dir}")




image 1/50 /content/dataset/test/163_jpg.rf.924ce8327dedd5bcbf91cf64536a1f27.jpg: 480x640 2 Vehicles, 98.2ms
image 2/50 /content/dataset/test/200_png.rf.054e44b617a4bfd3ca5e5ad5a120c05f.jpg: 384x640 3 Vehicles, 94.5ms
image 3/50 /content/dataset/test/30_jpg.rf.8443027a7bd896c17758ac346ff42670.jpg: 448x640 6 Vehicles, 98.0ms
image 4/50 /content/dataset/test/60_jpg.rf.3c041ee2d63dd927ef9236ab5807a450.jpg: 480x640 1 Vehicle, 18.5ms
image 5/50 /content/dataset/test/84_jpg.rf.ab914e066fef5d8a2ccf565228c0b9cb.jpg: 416x640 6 Vehicles, 76.5ms
image 6/50 /content/dataset/test/Dipto_-117_jpg.rf.a270357c66e39eafe7d19d093490fd05.jpg: 288x640 3 Vehicles, 57.3ms
image 7/50 /content/dataset/test/Dipto_-165_jpg.rf.fb72535aef8aea2b1eba5fcba990bdd8.jpg: 320x640 5 Vehicles, 56.8ms
image 8/50 /content/dataset/test/Dipto_-198_jpg.rf.59d76cb046858b70214df37cce91fdca.jpg: 384x640 2 Vehicles, 14.4ms
image 9/50 /content/dataset/test/Dipto_-250_jpg.rf.56da09b1cfde5753b55735dfa320a56f.jpg: 384x640 4 Vehicles, 1

In [21]:
import os

# Folder containing your test images
test_images_dir = '/content/dataset/test/'

# Folder to save predictions
output_dir = '/content/runs/predict/test_results/'
os.makedirs(output_dir, exist_ok=True)


In [22]:
# Predict on all test images
results = model.predict(
    source=test_images_dir,  # can be a folder or a single image
    save=True,               # save images with overlayed masks
    imgsz=640,               # resize for inference (optional)
    conf=0.25                # confidence threshold
)



image 1/50 /content/dataset/test/163_jpg.rf.924ce8327dedd5bcbf91cf64536a1f27.jpg: 480x640 2 Vehicles, 17.6ms
image 2/50 /content/dataset/test/200_png.rf.054e44b617a4bfd3ca5e5ad5a120c05f.jpg: 384x640 3 Vehicles, 14.4ms
image 3/50 /content/dataset/test/30_jpg.rf.8443027a7bd896c17758ac346ff42670.jpg: 448x640 6 Vehicles, 17.0ms
image 4/50 /content/dataset/test/60_jpg.rf.3c041ee2d63dd927ef9236ab5807a450.jpg: 480x640 1 Vehicle, 17.2ms
image 5/50 /content/dataset/test/84_jpg.rf.ab914e066fef5d8a2ccf565228c0b9cb.jpg: 416x640 6 Vehicles, 16.6ms
image 6/50 /content/dataset/test/Dipto_-117_jpg.rf.a270357c66e39eafe7d19d093490fd05.jpg: 288x640 3 Vehicles, 13.7ms
image 7/50 /content/dataset/test/Dipto_-165_jpg.rf.fb72535aef8aea2b1eba5fcba990bdd8.jpg: 320x640 5 Vehicles, 13.9ms
image 8/50 /content/dataset/test/Dipto_-198_jpg.rf.59d76cb046858b70214df37cce91fdca.jpg: 384x640 2 Vehicles, 14.4ms
image 9/50 /content/dataset/test/Dipto_-250_jpg.rf.56da09b1cfde5753b55735dfa320a56f.jpg: 384x640 4 Vehicles, 1

In [23]:
import shutil
shutil.make_archive('/content/test_results', 'zip', '/content/runs/predict/test_results/')
print("All predictions zipped at: /content/test_results.zip")


All predictions zipped at: /content/test_results.zip


In [24]:
import os

pred_dir = '/content/runs/predict/test_results/'
pred_files = os.listdir(pred_dir)
pred_files.sort()  # optional, to keep names in order

# Show first 5 files
pred_files[:5]

[]

In [28]:
from IPython.display import Image, display

# Pick the first predicted image
img_path = os.path.join(pred_dir, pred_files[0])
display(Image(filename=img_path))


IndexError: list index out of range

In [29]:
# Clone the ByteTrack repository
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack
!pip install -r requirements.txt


Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007 (from 1)
Receiving objects: 100% (2007/2007), 79.58 MiB | 28.42 MiB/s, done.
Resolving deltas: 100% (1158/1158), done.
/content/ByteTrack
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 94.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement onnxruntime==1.8.0 (from versions: 1.17.0, 1.17.1, 1.17.3, 1.18.0, 1.18.1, 1.19.0, 1.19.2, 1.20.0, 1.20.1, 1.21.0, 1.21.1, 1.22.0, 1.22.1)
ERROR: No matching distribution found for onnxruntime==1.8.0


In [30]:
import cv2
import os

img_folder = '/content/dataset/test/'
output_video = '/content/test_video.mp4'

images = sorted([os.path.join(img_folder, f) for f in os.listdir(img_folder) if f.endswith('.jpg')])
frame = cv2.imread(images[0])
height, width, _ = frame.shape
out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), 5, (width, height))

for img_path in images:
    frame = cv2.imread(img_path)
    out.write(frame)
out.release()
print("Video saved at:", output_video)


Video saved at: /content/test_video.mp4


In [33]:
# 🔹 Step 0: Install dependencies
!pip install loguru opencv-python scipy lappy

# 🔹 Step 1: Imports
from ultralytics import YOLO
import cv2
import numpy as np
import os
import sys

# Fix lap import for ByteTrack
sys.modules['lap'] = __import__('lappy')

# Import ByteTrack
from yolox.tracker.byte_tracker import BYTETracker

# 🔹 Step 2: Load YOLO-Seg model
model_path = '/content/runs/train/vehicle_pedestrian_yolov8/weights/best.pt'  # Update your path
yolo_model = YOLO(model_path)

# 🔹 Step 3: Convert test images to video (optional)
img_folder = '/content/dataset/test/'  # Folder of test images
output_video = '/content/test_video.mp4'

images = sorted([os.path.join(img_folder, f) for f in os.listdir(img_folder) if f.endswith('.jpg')])
frame = cv2.imread(images[0])
height, width, _ = frame.shape
out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), 5, (width, height))  # 5 FPS

for img_path in images:
    frame = cv2.imread(img_path)
    out.write(frame)
out.release()
print("Video saved at:", output_video)

# 🔹 Step 4: Initialize ByteTrack
tracker = BYTETracker(frame_rate=5)  # Match FPS of the video

# 🔹 Step 5: Run YOLO + ByteTrack tracking
cap = cv2.VideoCapture(output_video)
tracked_video_path = '/content/tracked_video.mp4'
out = cv2.VideoWriter(tracked_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 5, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO inference
    results = yolo_model.predict(frame, conf=0.25, verbose=False)
    dets = results[0].boxes.xyxy.cpu().numpy()  # bounding boxes
    scores = results[0].boxes.conf.cpu().numpy()  # confidence scores

    if len(dets) == 0:
        tracked_objs = []
    else:
        dets_bt = np.hstack((dets, scores[:, None]))  # [x1, y1, x2, y2, score]
        tracked_objs = tracker.update(dets_bt, [height, width], [height, width])

    # Draw tracked objects
    for obj in tracked_objs:
        x1, y1, x2, y2, track_id = map(int, obj[:5])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()
print("Tracked video saved at:", tracked_video_path)

# 🔹 Step 6: Display video in Colab
from IPython.display import Video
Video(tracked_video_path, embed=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.4/552.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 12.4 MB/s eta 0:00:00
  Created wheel for lappy: filename=lappy-2020.4.dev1-py3-none-any.whl size=55944 sha256=fc773312cf852d782ed364786ca11449235155bb222c5144353c8a8d08b22d78
  Stored in directory: /root/.cache/pip/wheels/8e/81/24/337b710b3f0b5feb2513b0d5341943a34cd286bf442b47a7ba
  

ModuleNotFoundError: No module named 'pyopencl'